fact_sales

In [0]:
%sql
create external table gold.fact_sales
using delta 
location 'abfss://gold@fordatabricks111.dfs.core.windows.net/fact_sales/';

In [0]:
query1 = """
SELECT
    sd.order_number,
    pr.product_key,
    cu.customer_key,
    sd.order_date,
    sd.ship_date,
    sd.due_date,
    sd.sales_amount,
    sd.quantity,
    sd.price
FROM databrickswp.silver.sales_details sd
LEFT JOIN gold.dim_products pr
    ON sd.product_number = pr.product_number
LEFT JOIN gold.dim_customers cu
    ON sd.customer_id = cu.customer_id;
"""
dffa = spark.sql(query)

dffa.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("databrickswp.gold.fact_sales")


In [0]:
dim_products

In [0]:
%sql
create external table gold.dim_products
using delta 
location 'abfss://gold@fordatabricks111.dfs.core.windows.net/dim_products/';

In [0]:
query2 = """
SELECT
    ROW_NUMBER() OVER (ORDER BY pn.start_date, pn.product_number) AS product_key, -- Surrogate key
    pn.product_id,
    pn.product_number,
    pn.product_name,
    pn.category_id,
    pc.category,
    pc.subcategory,
    pc.maintenance_flag,
    pn.product_line,
    pn.start_date
FROM databrickswp.silver.prd_info pn
LEFT JOIN databrickswp.silver.px_cat_g1v2 pc
    ON pn.category_id = pc.category_id
--WHERE pn.end_date IS NULL; -- Filter out all historical data
"""
dfpro = spark.sql(query2)


dfpro.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("gold.dim_products")


dim_customers

In [0]:
%sql
create external table gold.dim_customers
using delta 
location 'abfss://gold@fordatabricks111.dfs.core.windows.net/dim_customers/';

In [0]:
query3 = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_number,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM databrickswp.silver.cust_info ci
LEFT JOIN databrickswp.silver.cust_az12 ca
    ON ci.customer_number = ca.customer_number
LEFT JOIN databrickswp.silver.loc_a101 la
    ON ci.customer_number = la.customer_number
"""
dfcust = spark.sql(query3)


dfcust.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable("databrickswp.gold.dim_customers")